In [1]:
import socket
import ssl

In [2]:
EXAMPLE_URLS = [
    "http://example.org/",
    "http://browser.engineering/examples/example1-simple.html",
]

## Main


In [3]:
class URL:
    def __init__(self, url: str):
        self.scheme, url = url.split("://", 1)
        assert self.scheme in ["http", "https"]
        if self.scheme == "http":
            self.port = 80
        elif self.scheme == "https":
            self.port = 443
        if "/" not in url:
            url = url + "/"
        self.host, url = url.split("/", 1)
        if ":" in self.host:
            self.host, port = self.host.split(":", 1)
            self.port = int(port)
        self.path = "/" + url

    def request(self):
        s = socket.socket(
            family=socket.AF_INET,
            type=socket.SOCK_STREAM,
            proto=socket.IPPROTO_TCP,
        )
        s.connect((self.host, self.port))
        if self.scheme == "https":
            ctx = ssl.create_default_context()
            s = ctx.wrap_socket(s, server_hostname=self.host)
        request = "GET {} HTTP/1.0\r\n".format(self.path)
        request += "Host: {}\r\n".format(self.host)
        request += "\r\n"
        s.send(request.encode("utf8"))
        response = s.makefile("r", encoding="utf8", newline="\r\n")
        statusline = response.readline()
        version, status, explanation = statusline.split(" ", 2)
        response_headers = {}
        while True:
            line = response.readline()
            if line == "\r\n":
                break
            header, value = line.split(":", 1)
            response_headers[header.casefold()] = value.strip()
        assert "transfer-encoding" not in response_headers
        assert "content-encoding" not in response_headers
        body = response.read()
        s.close()
        return body

In [4]:
def show(body: str):
    in_tag = False
    for c in body:
        if c == "<":
            in_tag = True
        elif c == ">":
            in_tag = False
        elif not in_tag:
            print(c, end="")

In [5]:
def parse_html(body: str):
    in_tag = False
    for c in body:
        if c == "<":
            in_tag = True
        elif c == ">":
            in_tag = False
        elif not in_tag:
            yield c

In [6]:
def load(url: URL):
    body = url.request()
    show(body)

In [7]:
load(URL(EXAMPLE_URLS[0]))

Example Domainbody{background:#eee;width:60vw;margin:15vh auto;font-family:system-ui,sans-serif}h1{font-size:1.5em}div{opacity:0.8}a:link,a:visited{color:#348}Example DomainThis domain is for use in documentation examples without needing permission. Avoid use in operations.Learn more


In [8]:
load(URL("http://browser.engineering/http.html"))




  
  
  

  
  
  
  
  
  
  
  
  
  
  

  



  
  

  
  
  








Twitter ·
Blog ·
Discussions



  Chapter 1 of Web Browser Engineering.
  &lt;
  &gt;




  
    
    Buy a copy »
  
  
    Web Browser Engineering is now available.
    Buy a copy »
  





Connecting to a Server
Requesting Information
The
Server’s Response
Telnet in
Python
Request and Response
Displaying the HTML
Encrypted Connections
Summary
Outline
Exercises



A web browser displays information identified by a URL. And the first
step is to use that URL to connect to and download information from a
server somewhere on the internet.
Connecting to a Server
Browsing the internet starts with a URL,“URL” stands for “uniform
resource locator”, meaning that it is a portable (uniform) way to
identify web pages (resources) and also that it describes how to access
those files (locator). a short string that identifies a
particular web page that the browser should visit.


httpScheme://example.orgHostname/index.htmlP

In [9]:
load(URL(EXAMPLE_URLS[1]))


  
    This is a simple
    web page with some
    text in it.
  



## 1-1 HTTP/1.1


In [10]:
class URL_1_1:
    def __init__(self, url: str):
        self.scheme, url = url.split("://", 1)
        assert self.scheme in ["http", "https"]
        if self.scheme == "http":
            self.port = 80
        elif self.scheme == "https":
            self.port = 443
        if "/" not in url:
            url = url + "/"
        self.host, url = url.split("/", 1)
        if ":" in self.host:
            self.host, port = self.host.split(":", 1)
            self.port = int(port)
        self.path = "/" + url

    def request(self):
        s = socket.socket(
            family=socket.AF_INET,
            type=socket.SOCK_STREAM,
            proto=socket.IPPROTO_TCP,
        )
        s.connect((self.host, self.port))
        if self.scheme == "https":
            ctx = ssl.create_default_context()
            s = ctx.wrap_socket(s, server_hostname=self.host)
        headers = [
            f"GET {self.path} HTTP/1.1",
            f"Host: {self.host}",
            f"Connection: close",
            f"User-Agent: chomu-browser/0.1",
            "",
        ]
        request = "\r\n".join(headers) + "\r\n"
        s.send(request.encode("utf8"))
        response = s.makefile("r", encoding="utf8", newline="\r\n")
        statusline = response.readline()
        version, status, explanation = statusline.split(" ", 2)
        response_headers = {}
        while True:
            line = response.readline()
            if line == "\r\n":
                break
            header, value = line.split(":", 1)
            response_headers[header.casefold()] = value.strip()
        assert "transfer-encoding" not in response_headers
        assert "content-encoding" not in response_headers
        body = response.read()
        s.close()
        return body

    def load(self):
        body = self.request()
        show(body)


URL_1_1("http://example.org/").load()

Example Domainbody{background:#eee;width:60vw;margin:15vh auto;font-family:system-ui,sans-serif}h1{font-size:1.5em}div{opacity:0.8}a:link,a:visited{color:#348}Example DomainThis domain is for use in documentation examples without needing permission. Avoid use in operations.Learn more


In [11]:
URL_1_1("http://browser.engineering/examples/example1-simple.html").load()


  
    This is a simple
    web page with some
    text in it.
  



## 1-2 파일 URL


In [12]:
from pathlib import Path

In [13]:
FILE_URI = "file://" + str((Path(".").parent / "ch1" / "index.html").resolve())
print(FILE_URI)

file:///Users/chanhaeng/GitHub/web-browser-engineering-workbook/ch1/index.html


In [14]:
class URL_1_2:
    def __init__(self, url: str):
        self.scheme, url = url.split("://", 1)
        assert self.scheme in ["http", "https", "file"]
        match self.scheme:
            case "http":
                self.port = 80
            case "https":
                self.port = 443
            case "file":
                self.port = 0
        if "/" not in url:
            url = url + "/"
        self.host, url = url.split("/", 1)
        if self.scheme == "file" and self.host == "":
            self.host = "localhost"
        if ":" in self.host:
            self.host, port = self.host.split(":", 1)
            self.port = int(port)
        self.path = "/" + url

    def http(self):
        s = socket.socket(
            family=socket.AF_INET,
            type=socket.SOCK_STREAM,
            proto=socket.IPPROTO_TCP,
        )
        s.connect((self.host, self.port))
        if self.scheme == "https":
            ctx = ssl.create_default_context()
            s = ctx.wrap_socket(s, server_hostname=self.host)
        headers = [
            f"GET {self.path} HTTP/1.1",
            f"Host: {self.host}",
            f"Connection: close",
            f"User-Agent: chomu-browser/0.1",
            "",
        ]
        request = "\r\n".join(headers) + "\r\n"
        s.send(request.encode("utf8"))
        response = s.makefile("r", encoding="utf8", newline="\r\n")
        statusline = response.readline()
        version, status, explanation = statusline.split(" ", 2)
        response_headers = {}
        while True:
            line = response.readline()
            if line == "\r\n":
                break
            header, value = line.split(":", 1)
            response_headers[header.casefold()] = value.strip()
        assert "transfer-encoding" not in response_headers
        assert "content-encoding" not in response_headers
        body = response.read()
        s.close()
        return body

    def file(self):
        path = self.path
        with open(path, "r", encoding="utf8") as f:
            body = f.read()
        return body

    def request(self):
        match self.scheme:
            case "http" | "https":
                return self.http()
            case "file":
                return self.file()
        raise ValueError(f"unknown scheme: {self.scheme}")

    def load(self):
        body = self.request()
        match self.scheme:
            case "http" | "https":
                show(body)
            case "file":
                print(body)

In [15]:
URL_1_2(FILE_URI).load()

<html>
  <head></head>
  <body>
    <div>This is a simple</div>
    <div>web page with some</div>
    <span>text in it.</span>
  </body>
</html>



## 1-3 data: 스킴


In [16]:
import re
import base64
from urllib.parse import quote, unquote
from PIL import Image
from io import BytesIO

In [17]:
BASE64_IMG = "iVBORw0KGgoAAAANSUhEUgAAAC8AAAAeCAYAAAC8AUekAAACf0lEQVR4nM1YPVIbMRSWdrhCrkBNkQKncEHHHRicNlR2LgIVtIbhBincUVAkKVPnCtzByjxZn3h6vNU+2bvjfDPyrlbS6nv/K3v3fyCIvrcsOnHHJxXoZ3u/jp3u5qv55Z2bDoEIUQshuPPldXymNAficr0ifLt59kQg0hKz1SJef55e5GfQNgmBe1rrvZ+EY6G5CvkP2N6v1edJ0mKcvX/QCmbSIGEQIs49X17HZoUTrsX2Oszn4Qr+20IVTPRh+sJNrHsgZsaCqhmthfkmt+K5EdukbbqOp/mHx+E5843a1wK4D9yyLelzCFWNk2/XLFALWImhmGpJQ8ULUt6O+HR1pi54e/7jft897RbPN86/Xmbtz1YL9+t22Iq1dNlb8SSIbB9JC358/p4FACgYlcAvBCM3TW7jrZon8+fOIaQ1S4QkBO2BLPTl70ueJy3CBCg4V8mPRVpagCCtANSCWrpQzecnFSCIjMQFsQogU+WH6CZTUxuTOMiiEbgwRJCalpaTYLsCKMd4EKE6EsawAohrAPkkDDZOGdOpMdBJ4hhMK3N5npI4gVmBK9RrFoByOy27aBMPcZ3GtUFSIC1DCLQ4gAX8AwiEeYriwqHwWF0J5LHOiKEa5E8k8Tzy7u90E/o2JmKNLhXfZ5gXFAGKficLBGmdpSNMxj1/ljVpcQshvDd+mpiOgS2n9xgfqIKwENxKWkG4TG9RdH3nhsq3TVfTrLYJuRiVdAQNXI7IUduzLnjckBKsn84H/3tAbsZTKnelxkD18sQ1dHZo+iSWgc3iQ74vcE0OuIyE2YX3Ji8Cu7puj71M8BMTnxSdeSI7R+KPo2PDN84vKtzIXJrxD8LbWQUg1mYtAAAAAElFTkSuQmCC"
IMG_DATA_URI = f"data:image/png;base64,{BASE64_IMG}"

In [18]:
class URL_1_3:
    scheme: str
    host: str
    port: int
    data: str
    media_type: str
    is_base64: bool

    DATA_REGEX = re.compile(
        r"data:(?P<media_type>[\w/+.-]+)?(?P<base64>;base64)?,(?P<data>.*)"
    )

    def __init__(self, url: str):
        if data_match := URL_1_3.DATA_REGEX.match(url):
            data_info = self._parse_data_uri(data_match)
            self.scheme = data_info["scheme"]
            self.data = data_info["data"]
            self.media_type = data_info["media_type"]
            self.is_base64 = data_info["is_base64"]
            return
        self.scheme, url = url.split("://", 1)
        assert self.scheme in ["http", "https", "file"]
        match self.scheme:
            case "http":
                self.port = 80
            case "https":
                self.port = 443
        if "/" not in url:
            url = url + "/"
        self.host, url = url.split("/", 1)
        if self.scheme == "file" and self.host == "":
            self.host = "localhost"
        if ":" in self.host:
            self.host, port = self.host.split(":", 1)
            self.port = int(port)
        self.path = "/" + url

    @staticmethod
    def _parse_data_uri(data_match: re.Match):
        return {
            "scheme": "data",
            "data": data_match.group("data"),
            "media_type": data_match.group("media_type")
            or "text/plain;charset=US-ASCII",
            "is_base64": bool(data_match.group("base64")),
        }

    def _http(self):
        s = socket.socket(
            family=socket.AF_INET,
            type=socket.SOCK_STREAM,
            proto=socket.IPPROTO_TCP,
        )
        s.connect((self.host, self.port))
        if self.scheme == "https":
            ctx = ssl.create_default_context()
            s = ctx.wrap_socket(s, server_hostname=self.host)
        headers = [
            f"GET {self.path} HTTP/1.1",
            f"Host: {self.host}",
            f"Connection: close",
            f"User-Agent: chomu-browser/0.1",
            "",
        ]
        request = "\r\n".join(headers) + "\r\n"
        s.send(request.encode("utf8"))
        response = s.makefile("r", encoding="utf8", newline="\r\n")
        statusline = response.readline()
        version, status, explanation = statusline.split(" ", 2)
        response_headers = {}
        while True:
            line = response.readline()
            if line == "\r\n":
                break
            header, value = line.split(":", 1)
            response_headers[header.casefold()] = value.strip()
        assert "transfer-encoding" not in response_headers
        assert "content-encoding" not in response_headers
        body = response.read()
        s.close()
        return "".join(parse_html(body))

    def _file(self):
        path = self.path
        with open(path, "r", encoding="utf8") as f:
            body = f.read()
        return body

    def _data(self):
        if self.is_base64:
            body = base64.b64decode(self.data)
            if self.media_type.startswith("text/"):
                return body.decode("utf8")
            if self.media_type.startswith("image/"):
                return Image.open(BytesIO(body))
            return body
        else:
            return quote(self.data)

    def request(self):
        match self.scheme:
            case "http" | "https":
                return self._http()
            case "file":
                return self._file()
            case "data":
                return self._data()
        raise ValueError(f"unknown scheme: {self.scheme}")

    def load(self):
        body = self.request()
        display(body)


URL_1_3("data:text/plain,Hello, World!").load()
URL_1_3(IMG_DATA_URI).load()

'Hello%2C%20World%21'

## 1-4 HTML 엔티티(특수코드)


In [19]:
import html  # html.unescape

In [20]:
class URL_1_4:
    scheme: str
    host: str
    port: int
    data: str
    media_type: str
    is_base64: bool

    DATA_REGEX = re.compile(
        r"data:(?P<media_type>[\w/+.-]+)?(?P<base64>;base64)?,(?P<data>.*)"
    )

    def __init__(self, url: str):
        if data_match := URL_1_3.DATA_REGEX.match(url):
            data_info = self._parse_data_uri(data_match)
            self.scheme = data_info["scheme"]
            self.data = data_info["data"]
            self.media_type = data_info["media_type"]
            self.is_base64 = data_info["is_base64"]
            return
        self.scheme, url = url.split("://", 1)
        assert self.scheme in ["http", "https", "file"]
        match self.scheme:
            case "http":
                self.port = 80
            case "https":
                self.port = 443
        if "/" not in url:
            url = url + "/"
        self.host, url = url.split("/", 1)
        if self.scheme == "file" and self.host == "":
            self.host = "localhost"
        if ":" in self.host:
            self.host, port = self.host.split(":", 1)
            self.port = int(port)
        self.path = "/" + url

    @staticmethod
    def _parse_data_uri(data_match: re.Match):
        return {
            "scheme": "data",
            "data": data_match.group("data"),
            "media_type": data_match.group("media_type")
            or "text/plain;charset=US-ASCII",
            "is_base64": bool(data_match.group("base64")),
        }

    def _http(self):
        s = socket.socket(
            family=socket.AF_INET,
            type=socket.SOCK_STREAM,
            proto=socket.IPPROTO_TCP,
        )
        s.connect((self.host, self.port))
        if self.scheme == "https":
            ctx = ssl.create_default_context()
            s = ctx.wrap_socket(s, server_hostname=self.host)
        headers = [
            f"GET {self.path} HTTP/1.1",
            f"Host: {self.host}",
            f"Connection: close",
            f"User-Agent: chomu-browser/0.1",
            "",
        ]
        request = "\r\n".join(headers) + "\r\n"
        s.send(request.encode("utf8"))
        response = s.makefile("r", encoding="utf8", newline="\r\n")
        statusline = response.readline()
        version, status, explanation = statusline.split(" ", 2)
        response_headers = {}
        while True:
            line = response.readline()
            if line == "\r\n":
                break
            header, value = line.split(":", 1)
            response_headers[header.casefold()] = value.strip()
        assert "transfer-encoding" not in response_headers
        assert "content-encoding" not in response_headers
        body = response.read()
        s.close()
        return "".join(parse_html(body))

    def _file(self):
        path = self.path
        with open(path, "r", encoding="utf8") as f:
            body = f.read()
        return body

    def _data(self):
        if self.is_base64:
            body = base64.b64decode(self.data)
            if self.media_type.startswith("text/"):
                return body.decode("utf8")
            if self.media_type.startswith("image/"):
                return Image.open(BytesIO(body))
            return body
        else:
            return self.data

    def request(self):
        match self.scheme:
            case "http" | "https":
                return self._http()
            case "file":
                return self._file()
            case "data":
                return self._data()
        raise ValueError(f"unknown scheme: {self.scheme}")

    def load(self):
        body = self.request()
        display(html.unescape(body) if type(body) is str else body)


URL_1_4("data:text/plain,&lt;div&gt;").load()
URL_1_4(IMG_DATA_URI).load()

'<div>'

## 1-5 view-source 스킴


In [21]:
class URL_1_5:
    scheme: str
    host: str
    port: int
    data: str
    media_type: str
    is_base64: bool
    is_view_source: bool

    DATA_REGEX = re.compile(
        r"data:(?P<media_type>[\w/+.-]+)?(?P<base64>;base64)?,(?P<data>.*)"
    )

    def __init__(self, url: str):
        if data_match := URL_1_3.DATA_REGEX.match(url):
            data_info = self._parse_data_uri(data_match)
            self.scheme = data_info["scheme"]
            self.data = data_info["data"]
            self.media_type = data_info["media_type"]
            self.is_base64 = data_info["is_base64"]
            return
        self.is_view_source = url.startswith("view-source:")
        if self.is_view_source:
            url = url[len("view-source:") :]
        self.scheme, url = url.split("://", 1)
        assert self.scheme in ["http", "https", "file"]
        match self.scheme:
            case "http":
                self.port = 80
            case "https":
                self.port = 443
        if "/" not in url:
            url = url + "/"
        self.host, url = url.split("/", 1)
        if self.scheme == "file" and self.host == "":
            self.host = "localhost"
        if ":" in self.host:
            self.host, port = self.host.split(":", 1)
            self.port = int(port)
        self.path = "/" + url

    @staticmethod
    def _parse_data_uri(data_match: re.Match):
        return {
            "scheme": "data",
            "data": data_match.group("data"),
            "media_type": data_match.group("media_type")
            or "text/plain;charset=US-ASCII",
            "is_base64": bool(data_match.group("base64")),
        }

    def _http(self):
        s = socket.socket(
            family=socket.AF_INET,
            type=socket.SOCK_STREAM,
            proto=socket.IPPROTO_TCP,
        )
        s.connect((self.host, self.port))
        if self.scheme == "https":
            ctx = ssl.create_default_context()
            s = ctx.wrap_socket(s, server_hostname=self.host)
        headers = [
            f"GET {self.path} HTTP/1.1",
            f"Host: {self.host}",
            f"Connection: close",
            f"User-Agent: chomu-browser/0.1",
            "",
        ]
        request = "\r\n".join(headers) + "\r\n"
        s.send(request.encode("utf8"))
        response = s.makefile("r", encoding="utf8", newline="\r\n")
        statusline = response.readline()
        version, status, explanation = statusline.split(" ", 2)
        response_headers = {}
        while True:
            line = response.readline()
            if line == "\r\n":
                break
            header, value = line.split(":", 1)
            response_headers[header.casefold()] = value.strip()
        assert "transfer-encoding" not in response_headers
        assert "content-encoding" not in response_headers
        body = response.read()
        s.close()
        if self.is_view_source:
            return body
        return "".join(parse_html(body))

    def _file(self):
        path = self.path
        with open(path, "r", encoding="utf8") as f:
            body = f.read()
        return body

    def _data(self):
        if self.is_base64:
            body = base64.b64decode(self.data)
            if self.media_type.startswith("text/"):
                return body.decode("utf8")
            if self.media_type.startswith("image/"):
                return Image.open(BytesIO(body))
            return body
        else:
            return self.data

    def request(self):
        match self.scheme:
            case "http" | "https":
                return self._http()
            case "file":
                return self._file()
            case "data":
                return self._data()
        raise ValueError(f"unknown scheme: {self.scheme}")

    def load(self):
        body = self.request()
        display(html.unescape(body) if type(body) is str else body)


for url in EXAMPLE_URLS:
    URL_1_5("view-source:" + url).load()

'<!doctype html><html lang="en"><head><title>Example Domain</title><meta name="viewport" content="width=device-width, initial-scale=1"><style>body{background:#eee;width:60vw;margin:15vh auto;font-family:system-ui,sans-serif}h1{font-size:1.5em}div{opacity:0.8}a:link,a:visited{color:#348}</style><body><div><h1>Example Domain</h1><p>This domain is for use in documentation examples without needing permission. Avoid use in operations.<p><a href="https://iana.org/domains/example">Learn more</a></div></body></html>\n'

'<html>\n  <body>\n    <div>This is a simple</div>\n    <div>web page with some</div>\n    <span>text in it.</span>\n  </body>\n</html>\n'

In [22]:
EXAMPLE_URLS

['http://example.org/',
 'http://browser.engineering/examples/example1-simple.html']

## 1-6 Keep-alive


In [24]:
from typing import Literal
from io import BufferedReader

In [25]:
class HTTPConnection:
    socket: socket.SocketType
    connection: Literal["keep-alive"] | Literal["close"]
    headers: dict
    response: BufferedReader

    def __init__(self, url: str):
        self.scheme, url = url.split("://", 1)
        match self.scheme:
            case "http":
                self.port = 80
            case "https":
                self.port = 443
        self.host = url.split("/", 1)[0]
        if ":" in self.host:
            self.host, port = self.host.split(":", 1)
            self.port = int(port)
        self.open_socket()
        self.connect()

    def request(self, path: str = "/"):
        self.send_request(path)
        self.fetch_headers()
        return self.get_body()

    def open_socket(self):
        self._socket = socket.socket(
            family=socket.AF_INET,
            type=socket.SOCK_STREAM,
            proto=socket.IPPROTO_TCP,
        )

    def connect(self):
        self._socket.connect((self.host, self.port))
        if self.scheme == "https":
            ctx = ssl.create_default_context()
            self._socket = ctx.wrap_socket(self._socket, server_hostname=self.host)

    def send_request(self, path: str):
        headers = [
            f"GET {path} HTTP/1.1",
            f"Host: {self.host}",
            f"Connection: keep-alive",
            f"User-Agent: chomu-browser/0.1",
            "",
        ]
        request = "\r\n".join(headers) + "\r\n"
        self._socket.send(request.encode("utf8"))

    def fetch_headers(self):
        self.response = self._socket.makefile("rb")
        line = self.response.readline()
        version, status, explanation = line.decode("utf8").split(" ", 2)
        headers: dict = {
            "version": version,
            "status": status,
            "explanation": explanation,
        }
        while (line := self.response.readline().decode("utf8")) != "\r\n":
            header, value = line.split(":", 1)
            headers[header.casefold()] = value.strip()
        assert "transfer-encoding" not in headers
        assert "content-encoding" not in headers
        self.connection = headers.get("connection", "keep-alive").lower()
        self.headers = headers

    def get_body(self):
        content_length = int(self.headers.get("content-length", -1))
        if self.is_alive:
            return self.response.read(content_length).decode("utf8")
        raise RuntimeError("Connection is closed")

    @property
    def is_alive(self) -> bool:
        return self.connection == "keep-alive"

    def close(self):
        self._socket.close()

In [26]:
connection_by_host: dict[str, HTTPConnection] = {}


class URL_1_6:
    scheme: str
    host: str
    port: int
    path: str
    data: str
    media_type: str
    is_base64: bool
    is_view_source: bool
    connection: HTTPConnection

    DATA_REGEX = re.compile(
        r"data:(?P<media_type>[\w/+.-]+)?(?P<base64>;base64)?,(?P<data>.*)"
    )

    def __init__(self, url: str):
        if data_match := URL_1_3.DATA_REGEX.match(url):
            data_info = self._parse_data_uri(data_match)
            self.scheme = data_info["scheme"]
            self.data = data_info["data"]
            self.media_type = data_info["media_type"]
            self.is_base64 = data_info["is_base64"]
            return
        self.is_view_source = url.startswith("view-source:")
        if self.is_view_source:
            url = url[len("view-source:") :]
        if url.startswith("http://") or url.startswith("https://"):
            host, *path = url.split("://", 1)[1].split("/", 1)[0]
            if path:
                self.path = "/" + path[0]
            else:
                self.path = "/"
            if host not in connection_by_host:
                connection_by_host[host] = HTTPConnection(url)
            self.connection = connection_by_host[host]
            self.scheme = self.connection.scheme
            self.host = self.connection.host
            self.port = self.connection.port
            return
        self.scheme, url = url.split("://", 1)
        if self.scheme == "file" and self.host == "":
            self.host = "localhost"
        self.path = "/" + url

    @staticmethod
    def _parse_data_uri(data_match: re.Match):
        return {
            "scheme": "data",
            "data": data_match.group("data"),
            "media_type": data_match.group("media_type")
            or "text/plain;charset=US-ASCII",
            "is_base64": bool(data_match.group("base64")),
        }

    def _http(self):
        body = self.connection.request() or ""
        if self.is_view_source:
            return body
        return "".join(parse_html(body))

    def _file(self):
        path = self.path
        with open(path, "r", encoding="utf8") as f:
            body = f.read()
        return body

    def _data(self):
        if self.is_base64:
            body = base64.b64decode(self.data)
            if self.media_type.startswith("text/"):
                return body.decode("utf8")
            if self.media_type.startswith("image/"):
                return Image.open(BytesIO(body))
            return body
        else:
            return self.data

    def request(self):
        match self.scheme:
            case "http" | "https":
                return self._http()
            case "file":
                return self._file()
            case "data":
                return self._data()
        raise ValueError(f"unknown scheme: {self.scheme}")

    def load(self):
        body = self.request()
        display(html.unescape(body) if type(body) is str else body)


urls = [URL_1_6(url) for url in EXAMPLE_URLS]
for url in urls:
    url.load()

for conn in connection_by_host.values():
    conn.close()

'Example Domainbody{background:#eee;width:60vw;margin:15vh auto;font-family:system-ui,sans-serif}h1{font-size:1.5em}div{opacity:0.8}a:link,a:visited{color:#348}Example DomainThis domain is for use in documentation examples without needing permission. Avoid use in operations.Learn more\n'

'\n\n\n  \n  \n  \n\n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n\n  \n\n  \n\n\n\n\n  \n  \n  \n\n  Web Browser Engineering\n\n\n\n\n\n\n\nWeb Browser Engineering\nPavel Panchekha & Chris Harrelson\nTwitter ·\nBlog ·\nDiscussions\n\n\n\n\n\n\n\nIntroduction\nPart\n1: Loading Pages\nPart 2: Viewing Documents\nPart 3: Running\nApplications\nPart 4: Modern Browsers\n\n\n\nWeb browsers are ubiquitous, but how do they work? This book\nexplains, building a basic but complete web browser, from networking to\nJavaScript, in a couple thousand lines of Python.\n\n\n\nThe cover for Web Browser Engineering,\nfrom Oxford University Press\n\n\nBuy Web Browser Engineering\nPlease support us by buying Web Browser Engineering from a\nreseller like Bookshop.org,\nB&N,\nand Amazon. It’s currently $50 in\nthe US and £40 in the UK, with similar prices in many other countries.\nTranslations are coming soon!\n\n\nFollow this book’s blog, Mastodon, or Twitter for updates. There’s\na discussion\nforum for the

## 1-7 리다이렉트


In [27]:
REDIRECT_URLS = [
    "http://browser.engineering/redirect",
    "http://browser.engineering/redirect2",
    "http://browser.engineering/redirect3",
]

In [40]:
connection_by_host: dict[str, "HTTPConnection_1_7"] = {}


class HTTPConnection_1_7:
    socket: socket.SocketType
    connection: Literal["keep-alive"] | Literal["close"]
    headers: dict
    response: BufferedReader

    def __init__(self, url: str):
        self.scheme, url = url.split("://", 1)
        match self.scheme:
            case "http":
                self.port = 80
            case "https":
                self.port = 443
        self.host = url.split("/", 1)[0]
        if ":" in self.host:
            self.host, port = self.host.split(":", 1)
            self.port = int(port)
        self.open_socket()
        self.connect()

    def request(self, path: str = "/"):
        self.send_request(path)
        self.fetch_headers()
        while self.is_redirect():
            location: str = self.headers["location"]
            if location.startswith("http://") or location.startswith("https://"):
                return URL_1_7(location).request()
            self.send_request(location)
            self.fetch_headers()
        return self.get_body()

    def open_socket(self):
        self._socket = socket.socket(
            family=socket.AF_INET,
            type=socket.SOCK_STREAM,
            proto=socket.IPPROTO_TCP,
        )

    def connect(self):
        self._socket.connect((self.host, self.port))
        if self.scheme == "https":
            ctx = ssl.create_default_context()
            self._socket = ctx.wrap_socket(self._socket, server_hostname=self.host)

    def send_request(self, path: str):
        headers = [
            f"GET {path} HTTP/1.1",
            f"Host: {self.host}",
            f"Connection: keep-alive",
            f"User-Agent: chomu-browser/0.1",
            "",
        ]
        request = "\r\n".join(headers) + "\r\n"
        self._socket.send(request.encode("utf8"))

    def fetch_headers(self):
        self.response = self._socket.makefile("rb")
        line = self.response.readline()
        first = line.decode("utf8")
        version, status, explanation = first.split(" ", 2)
        headers: dict = {
            "version": version,
            "status": status,
            "explanation": explanation,
        }
        while (line := self.response.readline().decode("utf8")) != "\r\n":
            header, value = line.split(":", 1)
            headers[header.casefold()] = value.strip()
        assert "transfer-encoding" not in headers
        assert "content-encoding" not in headers
        self.connection = headers.get("connection", "keep-alive").lower()
        self.headers = headers

    def is_redirect(self) -> bool:
        return self.headers["status"] == "300"

    def get_body(self):
        content_length = int(self.headers.get("content-length", -1))
        if self.is_alive:
            return self.response.read(content_length).decode("utf8")
        raise RuntimeError("Connection is closed")

    @property
    def is_alive(self) -> bool:
        return self.connection == "keep-alive"

    def close(self):
        self._socket.close()


class URL_1_7:
    scheme: str
    host: str
    port: int
    path: str
    data: str
    media_type: str
    is_base64: bool
    is_view_source: bool
    connection: HTTPConnection_1_7

    DATA_REGEX = re.compile(
        r"data:(?P<media_type>[\w/+.-]+)?(?P<base64>;base64)?,(?P<data>.*)"
    )

    def __init__(self, url: str):
        if data_match := URL_1_3.DATA_REGEX.match(url):
            data_info = self._parse_data_uri(data_match)
            self.scheme = data_info["scheme"]
            self.data = data_info["data"]
            self.media_type = data_info["media_type"]
            self.is_base64 = data_info["is_base64"]
            return
        self.is_view_source = url.startswith("view-source:")
        if self.is_view_source:
            url = url[len("view-source:") :]
        if url.startswith("http://") or url.startswith("https://"):
            host, *path = url.split("://", 1)[1].split("/", 1)[0]
            if path:
                self.path = "/" + path[0]
            else:
                self.path = "/"
            if host not in connection_by_host:
                connection_by_host[host] = HTTPConnection_1_7(url)
            self.connection = connection_by_host[host]
            self.scheme = self.connection.scheme
            self.host = self.connection.host
            self.port = self.connection.port
            return
        self.scheme, url = url.split("://", 1)
        if self.scheme == "file" and self.host == "":
            self.host = "localhost"
        self.path = "/" + url

    @staticmethod
    def _parse_data_uri(data_match: re.Match):
        return {
            "scheme": "data",
            "data": data_match.group("data"),
            "media_type": data_match.group("media_type")
            or "text/plain;charset=US-ASCII",
            "is_base64": bool(data_match.group("base64")),
        }

    def _http(self):
        body = self.connection.request() or ""
        if self.is_view_source:
            return body
        return "".join(parse_html(body))

    def _file(self):
        path = self.path
        with open(path, "r", encoding="utf8") as f:
            body = f.read()
        return body

    def _data(self):
        if self.is_base64:
            body = base64.b64decode(self.data)
            if self.media_type.startswith("text/"):
                return body.decode("utf8")
            if self.media_type.startswith("image/"):
                return Image.open(BytesIO(body))
            return body
        else:
            return self.data

    def request(self):
        match self.scheme:
            case "http" | "https":
                return self._http()
            case "file":
                return self._file()
            case "data":
                return self._data()
        raise ValueError(f"unknown scheme: {self.scheme}")

    def load(self):
        body = self.request()
        display(html.unescape(body) if type(body) is str else body)


urls = [URL_1_7(url) for url in REDIRECT_URLS]
for url in urls:
    url.load()

for conn in connection_by_host.values():
    conn.close()

'\n\n\n  \n  \n  \n\n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n\n  \n\n  \n\n\n\n\n  \n  \n  \n\n  Web Browser Engineering\n\n\n\n\n\n\n\nWeb Browser Engineering\nPavel Panchekha & Chris Harrelson\nTwitter ·\nBlog ·\nDiscussions\n\n\n\n\n\n\n\nIntroduction\nPart\n1: Loading Pages\nPart 2: Viewing Documents\nPart 3: Running\nApplications\nPart 4: Modern Browsers\n\n\n\nWeb browsers are ubiquitous, but how do they work? This book\nexplains, building a basic but complete web browser, from networking to\nJavaScript, in a couple thousand lines of Python.\n\n\n\nThe cover for Web Browser Engineering,\nfrom Oxford University Press\n\n\nBuy Web Browser Engineering\nPlease support us by buying Web Browser Engineering from a\nreseller like Bookshop.org,\nB&N,\nand Amazon. It’s currently $50 in\nthe US and £40 in the UK, with similar prices in many other countries.\nTranslations are coming soon!\n\n\nFollow this book’s blog, Mastodon, or Twitter for updates. There’s\na discussion\nforum for the

'\n\n\n  \n  \n  \n\n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n\n  \n\n  \n\n\n\n\n  \n  \n  \n\n  Web Browser Engineering\n\n\n\n\n\n\n\nWeb Browser Engineering\nPavel Panchekha & Chris Harrelson\nTwitter ·\nBlog ·\nDiscussions\n\n\n\n\n\n\n\nIntroduction\nPart\n1: Loading Pages\nPart 2: Viewing Documents\nPart 3: Running\nApplications\nPart 4: Modern Browsers\n\n\n\nWeb browsers are ubiquitous, but how do they work? This book\nexplains, building a basic but complete web browser, from networking to\nJavaScript, in a couple thousand lines of Python.\n\n\n\nThe cover for Web Browser Engineering,\nfrom Oxford University Press\n\n\nBuy Web Browser Engineering\nPlease support us by buying Web Browser Engineering from a\nreseller like Bookshop.org,\nB&N,\nand Amazon. It’s currently $50 in\nthe US and £40 in the UK, with similar prices in many other countries.\nTranslations are coming soon!\n\n\nFollow this book’s blog, Mastodon, or Twitter for updates. There’s\na discussion\nforum for the

'\n\n\n  \n  \n  \n\n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n  \n\n  \n\n  \n\n\n\n\n  \n  \n  \n\n  Web Browser Engineering\n\n\n\n\n\n\n\nWeb Browser Engineering\nPavel Panchekha & Chris Harrelson\nTwitter ·\nBlog ·\nDiscussions\n\n\n\n\n\n\n\nIntroduction\nPart\n1: Loading Pages\nPart 2: Viewing Documents\nPart 3: Running\nApplications\nPart 4: Modern Browsers\n\n\n\nWeb browsers are ubiquitous, but how do they work? This book\nexplains, building a basic but complete web browser, from networking to\nJavaScript, in a couple thousand lines of Python.\n\n\n\nThe cover for Web Browser Engineering,\nfrom Oxford University Press\n\n\nBuy Web Browser Engineering\nPlease support us by buying Web Browser Engineering from a\nreseller like Bookshop.org,\nB&N,\nand Amazon. It’s currently $50 in\nthe US and £40 in the UK, with similar prices in many other countries.\nTranslations are coming soon!\n\n\nFollow this book’s blog, Mastodon, or Twitter for updates. There’s\na discussion\nforum for the